## Define module in wihch `#export` tag will save the code in `src`

In [1]:
#default_exp rolling

## Import modules that are only used in documentation and nbdev related (not going to src)

In [2]:
#hide
from nbdev.showdoc import *

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..') #appends project root to path in order to import project packages since `noteboks_dev` is not on the root

#DO NOT EDIT

In [3]:
#hide

#Internal Imports
#imports that are going to be used only during development and are not intended to be loaded inside the generated modules.
#for example: use imported modules to generate graphs for documentation, but lib is unused in actual package

#import ...

# rolling

## Dev comments

### TODOs

- [X] TODO: do something
- [ ] TODO: do something else

### <comment section 2>

## Code Session

### External Iimports
> imports that are intended to be loaded in the actual modules e.g.: module dependencies

In [4]:
#export
from functools import reduce, partial
import os
import datetime as dt
from tqdm import tqdm
from warnings import warn

import pandas as pd
import numpy as np
import numba

from dask import dataframe as dd
from dask import delayed
from dask.diagnostics import ProgressBar


### utils -

### Create historical "open invoices" features

In [5]:
#export

def _get_index_rolling_windows(rolling_obj):
    '''
    get positional indexes of rows of each rolling window
    '''    
    
    if hasattr(rolling_obj, '_selection'):        
        previous_selection = getattr(rolling_obj, '_selection')
    else:
        previous_selection = None
    
    INDEX_LIST = []
    #define function to append values to global INDEX_LIST since rolling apply won't let return arrays
    def f(x): INDEX_LIST.append(x.astype(int)); return 0    
    assert '__indexer__' not in rolling_obj.obj.columns, 'DataFrame should not contain any col with "__indexer__" name'
    rolling_obj.obj = rolling_obj.obj.assign(__indexer__ = np.arange(len(rolling_obj.obj)), inplace = True)
    rolling_obj._selection = '__indexer__'
    rolling_obj.apply(f, raw = True)
    rolling_obj.obj = rolling_obj.obj.drop(columns = ['__indexer__'])
    
    delattr(rolling_obj, '_selection')
    
    if not previous_selection is None:
        setattr(rolling_obj, '_selection', previous_selection)
    
    return INDEX_LIST


def _apply_custom_rolling(rolling_obj, func, raw = True, engine = 'numpy', *args, **kwargs):
    
    engines = {
        'numpy':_rolling_apply_custom_agg_numpy,
        'pandas':_rolling_apply_custom_agg_pandas,
        'numba':_rolling_apply_custom_agg_numpy_jit
    }
    _rolling_apply = engines[engine]
    
    indexes = _get_index_rolling_windows(rolling_obj)    
    if hasattr(rolling_obj, '_selection'):
        if getattr(rolling_obj, '_selection') is None:
            values = _rolling_apply(rolling_obj.obj, indexes, func, *args, **kwargs)    
        
        values = _rolling_apply(rolling_obj.obj[rolling_obj._selection], indexes, func, *args, **kwargs)
    else:
        values = _rolling_apply(rolling_obj.obj, indexes, func, *args, **kwargs)

    return values
    


def _rolling_apply_custom_agg_numpy_jit(df, indexes, func):
    '''
    applies some aggregation function over groups defined by index.
    groups are numpy arrays
    '''
    
    dfv = df.values
    # template of output to create empty array
    #use this for jit version
    shape = np.array(func(dfv[:1])).shape    
    #d = [np.empty(*shape) for _ in  range(len(indexes))]    
    result_array = np.empty((len(indexes),*shape))    
    
    @numba.jit(forceobj=True)
    def _roll_apply(dfv, indexes, func, result_array):
        for i in np.arange(len(indexes)):
            data = dfv[indexes[i]]                
            if len(data) > 0:
                result = func(data)
                result_array[i] = result
            else:                
                result = np.empty(shape)
                
            
        return result_array
    
    return _roll_apply(dfv, indexes, func, result_array)

    
def _rolling_apply_custom_agg_numpy(df, indexes, func, *args, **kwargs):
    '''
    applies some aggregation function over groups defined by index.
    groups are numpy arrays
    '''
    
    dfv = df.values    
    d = [[] for _ in range(len(indexes))]    
    for i in tqdm(range(len(indexes))):
        data = dfv[indexes[i]]                
        if len(data) > 0:
            result = func(data, *args, **kwargs)        
            d[i] = result
    
    return d

def _rolling_apply_custom_agg_pandas(df, indexes, func, *args, **kwargs):
    '''
    applies some aggregation function over groups defined by index.
    groups are pandas dataframes
    '''

    # template of output to create empty array
    d = [[] for _ in range(len(indexes))]        
    
    for i in tqdm(range(len(indexes))):
        data = df.iloc[indexes[i]]                
        if len(data) > 0:
            result = func(data, *args, **kwargs)        
            d[i] = result
    
    return pd.concat(d)

### Generic Rolling + resample features

In [21]:
#export
def _make_rolling_groupby_object(df, group_columns, date_column):
    '''
    helping function to make computational graph creation faster
    '''
    groupby_object = df.set_index(date_column).groupby(group_columns)

    return groupby_object

def make_generic_rolling_features(
    df,
    calculate_columns,
    group_columns,
    date_column,
    suffix = None,
    rolling_operation = 'mean',
    window = '60D',
    min_periods=None,
    center=False,
    win_type=None,
    on=None,
    axis=0,
    closed=None,
    **rolling_operation_kwargs
):
    '''
    make generic/custom rolling opeartion for a given column, grouped by customer, having Data de Emissao as date index
    if calculate cols is None, than use all cols

    Parameters
    ----------

    df: DataFrame
        DataFrame to make rolling features over

    calculate_columns: list of str
        list of columns to perform rolling_operation over

    group_columns: list of str
        list of columns passed to GroupBy operator prior to rolling

    date_column: str
        datetime column to roll over

    suffix: Str
        suffix for features names

    rolling_operation: Str of aggregation function, deafult = "mean"
        str representing groupby object method, such as mean, var, quantile ...

    window:
        DataFrameGroupBy.Rolling parameter. please refer to documentation

    min_periods:
        DataFrameGroupBy.Rolling parameter. please refer to documentation

    center:
        DataFrameGroupBy.Rolling parameter. please refer to documentation

    win_type:
        DataFrameGroupBy.Rolling parameter. please refer to documentation

    on:
        DataFrameGroupBy.Rolling parameter. please refer to documentation

    axis:
        DataFrameGroupBy.Rolling parameter. please refer to documentation

    closed:
        DataFrameGroupBy.Rolling parameter. please refer to documentation

    rolling_operation_kwargs:
        key word arguments passed to rolling_operation

    Returns
    -------
    DataFrame with the new calculated features
    '''

    assert group_columns.__class__ in (set, tuple, list), 'group_columns type should be one of (tuple, list, set), not {group_columns.__class__}'
    if calculate_columns is None:
        calculate_columns = [i for i in df.columns if not i in [*group_columns, date_column]]

    keep_columns = [*group_columns, date_column, *calculate_columns]

    if not isinstance(df,(
        dd.groupby.DataFrameGroupBy,
        pd.core.groupby.generic.DataFrameGroupBy,
        pd.core.groupby.generic.SeriesGroupBy,
        dd.groupby.SeriesGroupBy
    )):

        df = _make_rolling_groupby_object(df, group_columns, date_column)

    if isinstance(df, (pd.core.groupby.generic.DataFrameGroupBy, pd.core.groupby.generic.SeriesGroupBy)):


        df = getattr(
            df[calculate_columns]
            .rolling(
                window = window,
                min_periods=min_periods,
                center=center,
                win_type=win_type,
                on=on,
                axis=axis,
                closed=closed
            ),
            rolling_operation,

        )(**rolling_operation_kwargs).reset_index()

    else: #syntax for dask groupby rolling

        df = df[calculate_columns].apply(
                lambda x: getattr(
                    x.sort_index().rolling(
                        window = window,
                        min_periods=min_periods,
                        center=center,
                        win_type=win_type,
                        on=on,
                        axis=axis,
                        closed=closed
                    ),
                    rolling_operation,
                )(**rolling_operation_kwargs).reset_index()
            #meta = meta, #works only for float rolling

            ).reset_index().drop(columns = [f'level_{len(group_columns)}']) #drop unwanted "level_n" cols


    if not suffix:

        df.columns = [
            f'{col}__rolling_{rolling_operation}_{window}_{str(rolling_operation_kwargs)}'
            if not col in (*group_columns, date_column) else col
            for col in df.columns
        ]
    else:
        df.columns = [
            f'{col}__rolling_{window}_{suffix}'
            if not col in (*group_columns, date_column) else col
            for col in df.columns
        ]

    return df

def _make_shift_resample_groupby_object(df, group_columns, date_column,freq, n_periods_shift):

    groupby_object = (
            df
            .assign(**{date_column:df[date_column] + pd.Timedelta(n_periods_shift,freq)}) #shift
            .set_index(date_column)
            .groupby([*group_columns, pd.Grouper(freq = freq)])
        )
    return groupby_object

def make_generic_resampling_and_shift_features(
    df, calculate_columns, group_columns, date_column, freq = 'm',
    agg = 'last', n_periods_shift = 0, assert_frequency = False, suffix = '',**agg_kwargs
):

    '''
    makes generic resamples (aggregates by time frequency) on column.
    shifts one period up to avoid information leakage.
    Doing this through this function, although imposing some limitations to resampling periods, is much more efficient than
    pandas datetime-set_index + groupby + resampling.

    Parameters
    ----------

    df: DataFrame
        DataFrame to make rolling features over

    calculate_columns: list of str
        list of columns to perform rolling_operation over

    group_columns: list of str
        list of columns passed to GroupBy operator prior to rolling

    date_column: str
        datetime column to roll over

    freq: valid pandas freq str:
        frequency to resample data

    agg: Str of aggregation function, deafult = "last"
        str representing groupby object method, such as mean, var, last ...

    n_periods_shift: int
        number of periods to perform the shift opeartion. shifting is important after aggregation to avoid information leakage
        e.g. assuming you have the information of the end of the month in the beggining of the month.

    assert_frequency: bool, default = False
        resamples data to match freq, using foward fill method for
        missing values

    suffix: Str
        suffix for features names

    agg_kwargs:
        key word arguments passed to agg

    Returns
    -------
    DataFrame with the new calculated features
    '''

    if calculate_columns is None:
        calculate_columns = [i for i in df.columns if not i in [*group_columns, date_column]]

    keep_columns = [*group_columns, date_column, *calculate_columns]

    df = (
        df
        .assign(**{date_column:df[date_column] + pd.Timedelta(n_periods_shift,freq)}) #shift
        .set_index(date_column)
        .groupby([*group_columns, pd.Grouper(freq = freq)])
    )


    if isinstance(agg, str):
        df = getattr(df[calculate_columns], agg)(**agg_kwargs)
    else:
        df = df[calculate_columns].apply(lambda x: agg(x,**agg_kwargs))


    if not suffix:
        df.columns = [f'{i}__{str(agg)}_{str(agg_kwargs)}' for i in df.columns]
    else:
        df.columns = [f'{i}__{suffix}' for i in df.columns]

    #create new shifted date_col
    #df.loc[:, date_column] = date_col_values


    if assert_frequency:
        df = df.reset_index()
        df = df.set_index(date_column).groupby(group_columns).resample(freq).fillna(method = 'ffill')


    resetable_indexes = list(set(df.index.names) - set(df.columns))
    df = df.reset_index(level = resetable_indexes)
    df = df.reset_index(drop = True)

    return df


def create_rolling_resampled_features(
    df,
    calculate_columns,
    group_columns,
    date_column,
    extra_columns = [],
    n_periods_shift = 1,
    rolling_first = True,
    rolling_operation = 'mean',
    window = '60D',
    resample_freq = 'm',
    resample_agg = 'last',
    assert_frequency = False,
    rolling_suffix = '',
    resample_suffix = '',
    min_periods=None,
    center=False,
    win_type=None,
    on=None,
    axis=0,
    closed=None,
    rolling_operation_kwargs = {},
    resample_agg_kwargs = {}
):
    '''
    calculates rolling features groupwise, than resamples according to resample period.
    calculations can be done the other way arround if rolling_first is set to False

    Parameters
    ----------


    df: DataFrame
        DataFrame to make rolling features over

    calculate_columns: list of str
        list of columns to perform rolling_operation over

    group_columns: list of str
        list of columns passed to GroupBy operator prior to rolling

    date_column: str
        datetime column to roll over

    extra_columns: list of str
        list of extra columns to be passed to the final dataframe without aggregation (takes the last values, assumes they're constant along groupby).
        usefull to pass merge keys

    n_periods_shift: int
        number of periods to perform the shift opeartion. shifting is important after aggregation to avoid information leakage
        e.g. assuming you have the information of the end of the month in the beggining of the month.

    rolling_first: bool, deafult = True
        whether to perform rolling before resampling, or the other way arround

    rolling_operation: Str of aggregation function, deafult = "mean"
        str representing groupby object method, such as mean, var, quantile ...

    window:
        DataFrameGroupBy.Rolling parameter. please refer to documentation

    resample_freq: valid pandas freq str:
        frequency to resample data

    resample_agg: Str of aggregation function, deafult = "last"
        str representing groupby object method, such as mean, var, last ...

    assert_frequency: bool, default = False
        resamples data to match freq, using foward fill method for
        missing values

    rolling_suffix: Str
        suffix for the rolling part of features names

    resample_suffix: Str
        suffix for the resample part of features names

    min_periods:
        DataFrameGroupBy.Rolling parameter. please refer to documentation

    center:
        DataFrameGroupBy.Rolling parameter. please refer to documentation

    win_type:
        DataFrameGroupBy.Rolling parameter. please refer to documentation

    on:
        DataFrameGroupBy.Rolling parameter. please refer to documentation

    axis:
        DataFrameGroupBy.Rolling parameter. please refer to documentation

    closed:
        DataFrameGroupBy.Rolling parameter. please refer to documentation

    rolling_operation_kwargs: dict
        key word arguments passed to rolling_operation

    resample_agg_kwargs: dict
        key word arguments passed to resample_agg
    '''

    if rolling_first:

        features_df = make_generic_rolling_features(
            df,
            calculate_columns = calculate_columns,
            group_columns = group_columns,
            date_column = date_column,
            suffix = rolling_suffix,
            rolling_operation = rolling_operation,
            window = window,
            min_periods=min_periods,
            center=center,
            win_type=win_type,
            on=on,
            axis=axis,
            closed=closed,
            **rolling_operation_kwargs
        )


        if extra_columns:

            features_df = features_df.merge(
                df[extra_columns + group_columns + [date_column]],
                how = 'left',
                left_on = group_columns + [date_column],
                right_on = group_columns + [date_column]
            )


        features_df = make_generic_resampling_and_shift_features(
            features_df,
            calculate_columns = None,
            date_column = date_column,
            group_columns = group_columns,
            freq = resample_freq,
            agg = resample_agg,
            assert_frequency = assert_frequency,
            suffix = resample_suffix,
            n_periods_shift = n_periods_shift,
        )

    else:

        features_df = make_generic_resampling_and_shift_features(
            df,
            calculate_columns = calculate_columns,
            date_column = date_column,
            group_columns = group_columns,
            freq = resample_freq,
            agg = resample_agg,
            assert_frequency = assert_frequency,
            suffix = resample_suffix,
            n_periods_shift = n_periods_shift,
        )


        features_df = features_df.merge(
            df[extra_columns + group_columns + [date_column]],
            how = 'left',
            left_on = group_columns + [date_column],
            right_on = group_columns + [date_column]
        )

        features_df = make_generic_rolling_features(
            features_df,
            calculate_columns = None,
            group_columns = group_columns,
            date_column = date_column,
            suffix = rolling_suffix,
            rolling_operation = rolling_operation,
            window = window,
            min_periods=min_periods,
            center=center,
            win_type=win_type,
            on=on,
            axis=axis,
            closed=closed,
            **rolling_operation_kwargs
        )



    return features_df


## Experimentation session and usage examples

In [11]:
import pandas as pd
import dask.dataframe as dd

covid_data = pd.read_csv(
    r'.\datasets\covid_19_data.csv',
    parse_dates = ['ObservationDate']
)

covid_data

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,2020-01-22,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,2020-01-22,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,2020-01-22,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,2020-01-22,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,2020-01-22,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
285302,285303,2021-05-02,Zaporizhia Oblast,Ukraine,2021-05-03 04:20:39,96531.0,1919.0,78700.0
285303,285304,2021-05-02,Zeeland,Netherlands,2021-05-03 04:20:39,26045.0,233.0,0.0
285304,285305,2021-05-02,Zhejiang,Mainland China,2021-05-03 04:20:39,1344.0,1.0,1322.0
285305,285306,2021-05-02,Zhytomyr Oblast,Ukraine,2021-05-03 04:20:39,84641.0,1597.0,68529.0


In [20]:
make_generic_rolling_features(
    covid_data, 
    calculate_columns = ['Deaths','Confirmed'], 
    group_columns = ['Country/Region'],
    date_column = 'ObservationDate',
    rolling_operation = 'mean',
    window = '7D',
    suffix = ''
    
)

,Country/Region,ObservationDate,Deaths__rolling_mean_7D_{},Confirmed__rolling_mean_7D_{}
0,Azerbaijan,2020-02-28,0.0,1.000000
1,"('St. Martin',)",2020-03-10,0.0,2.000000
2,Afghanistan,2020-02-24,0.0,1.000000
3,Afghanistan,2020-02-25,0.0,1.000000
4,Afghanistan,2020-02-26,0.0,1.000000
...,...,...,...,...
285302,occupied Palestinian territory,2020-03-12,0.0,8.333333
285303,occupied Palestinian territory,2020-03-14,0.0,6.250000
285304,occupied Palestinian territory,2020-03-15,0.0,5.000000
285305,occupied Palestinian territory,2020-03-16,0.0,4.166667


In [23]:
make_generic_resampling_and_shift_features(
    covid_data, 
    calculate_columns = ['Deaths','Confirmed'], 
    group_columns = ['Country/Region'],
    date_column = 'ObservationDate',
    agg = 'mean',
    freq = 'W',
    suffix = '',
    assert_frequency = True
)

,ObservationDate,Country/Region,Deaths__mean_{},Confirmed__mean_{}
0,2020-03-01,Azerbaijan,0.000000,1.000000
1,2020-03-15,"('St. Martin',)",0.000000,2.000000
2,2020-03-01,Afghanistan,0.000000,1.000000
3,2020-03-08,Afghanistan,0.000000,3.428571
4,2020-03-15,Afghanistan,0.000000,11.714286
...,...,...,...,...
11778,2021-04-18,Zimbabwe,1548.428571,37487.428571
11779,2021-04-25,Zimbabwe,1555.142857,37989.571429
11780,2021-05-02,Zimbabwe,1566.000000,38212.857143
11781,2020-03-15,occupied Palestinian territory,0.000000,5.000000


In [25]:
create_rolling_resampled_features(
    covid_data, 
    calculate_columns = ['Deaths','Confirmed'], 
    group_columns = ['Country/Region'],
    date_column = 'ObservationDate',
    rolling_operation = 'mean',
    window = '15D',
    resample_freq = 'W'
)

,Country/Region,ObservationDate,Deaths__rolling_mean_15D_{}__last_{},Confirmed__rolling_mean_15D_{}__last_{}
0,Azerbaijan,2020-03-08,0.000000,1.000000
1,"('St. Martin',)",2020-03-22,0.000000,2.000000
2,Afghanistan,2020-03-08,0.000000,1.000000
3,Afghanistan,2020-03-15,0.000000,2.214286
4,Afghanistan,2020-03-22,0.000000,7.133333
...,...,...,...,...
11760,Zimbabwe,2021-04-25,1539.600000,37265.266667
11761,Zimbabwe,2021-05-02,1550.866667,37708.466667
11762,Zimbabwe,2021-05-09,1560.066667,38077.866667
11763,occupied Palestinian territory,2020-03-22,0.000000,5.000000


## Define jitted agg func to pass to engine = 'numba'

In [12]:
@numba.jit
def jit_sum(x):    
    return np.sum(x, axis = 0)

def jit_correlation(x):
    if x.shape[0] > 1:
        r = np.correlate(x[:,0],x[:,1],)
    else:
        r = np.nan
    return r

## Run for each 

In [13]:
brazil_data = covid_data.query('`Country/Region` == "Brazil"')
brazil_data = brazil_data.groupby(['ObservationDate','Province/State'])[['Confirmed','Deaths','Recovered']].sum().reset_index()
brazil_data = brazil_data.set_index('ObservationDate').groupby(['Province/State']).resample('D').fillna('ffill').reset_index(level = 'Province/State', drop = True)
brazil_data = brazil_data.query('`Province/State` != "Unknown"')
new_cases = brazil_data.groupby('Province/State')[['Confirmed','Deaths','Recovered']].diff()
new_cases.columns = ['new_'+i for i in ['Confirmed','Deaths','Recovered']]
brazil_data = pd.concat([brazil_data, new_cases], axis = 1)


In [14]:
brazil_data.query('`Province/State` == "Amazonas"')[['new_Confirmed','new_Deaths','new_Recovered']].apply(lambda x: (x-x.mean())/x.std())

,new_Confirmed,new_Deaths,new_Recovered
ObservationDate,,,
2020-05-20,NaN,NaN,NaN
2020-05-21,0.861867,0.671711,-0.354120
2020-05-22,0.872312,0.423565,-0.354120
2020-05-23,0.993743,1.068745,-0.354120
2020-05-24,0.081055,-0.444947,-0.354120
...,...,...,...
2021-04-28,-0.481703,-0.171986,0.360157
2021-04-29,-0.400750,-0.097542,-0.354120
2021-04-30,-0.536543,-0.494576,0.104892


In [15]:

grouper = covid_data.sample(10000).set_index('ObservationDate').groupby('Country/Region').rolling('30D')[['Confirmed','Deaths']]

%timeit -r 1 -n 1 _apply_custom_rolling(grouper, jit_correlation, engine = 'numba')
%timeit -r 1 -n 1 _apply_custom_rolling(grouper, jit_correlation, engine = 'numpy')

669 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


100%|████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 150999.72it/s]

105 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [16]:
_apply_custom_rolling(grouper, lambda x: np.corrcoef(x, rowvar = False).flatten(), engine = 'numpy')

  0%|                                                                                        | 0/10000 [00:00<?, ?it/s]c:\programdata\miniconda3\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\programdata\miniconda3\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
 11%|████████                                                                  | 1090/10000 [00:00<00:00, 10800.89it/s]c:\programdata\miniconda3\lib\site-packages\numpy\lib\function_base.py:2640: RuntimeWarning: invalid value encountered in true_divide
  return c / c
100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 12323.85it/s]


[array([1.]),
 array([1.]),
 array([1.]),
 array([1., 1., 1., 1.]),
 array([1.]),
 array([1., 1., 1., 1.]),
 array([1.        , 0.99978272, 0.99978272, 1.        ]),
 array([1.        , 0.99877457, 0.99877457, 1.        ]),
 array([1.        , 0.99214107, 0.99214107, 1.        ]),
 array([1.        , 0.98400251, 0.98400251, 1.        ]),
 array([1.        , 0.98808645, 0.98808645, 1.        ]),
 array([1.        , 0.98856861, 0.98856861, 1.        ]),
 array([1.        , 0.98694846, 0.98694846, 1.        ]),
 array([1.        , 0.98926599, 0.98926599, 1.        ]),
 array([1.        , 0.99041735, 0.99041735, 1.        ]),
 array([1.        , 0.98957986, 0.98957986, 1.        ]),
 array([1.       , 0.9889122, 0.9889122, 1.       ]),
 array([1.        , 0.98998611, 0.98998611, 1.        ]),
 array([1.        , 0.99031207, 0.99031207, 1.        ]),
 array([1.        , 0.98907071, 0.98907071, 1.        ]),
 array([1.]),
 array([1., 1., 1., 1.]),
 array([1.        , 0.99908348, 0.99908348, 

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()